# Calling Tools with LangGraph

Relevant documentation links:
- [LangChain agents](https://docs.langchain.com/oss/python/langchain/agents)
- [LangGraph for Python](https://docs.langchain.com/oss/python/langgraph/overview)

In [ ]:
import os
import io
import contextlib

from dotenv import load_dotenv

load_dotenv()

In [ ]:
BRIGHT_DATA_API_KEY = os.environ.get("BRIGHT_DATA_API_KEY")

assert BRIGHT_DATA_API_KEY is not None

In [ ]:
GOOGLE_GEMINI_API_KEY = os.environ.get("GOOGLE_GEMINI_API_KEY")

assert GOOGLE_GEMINI_API_KEY is not None

In [ ]:
from langchain_brightdata import BrightDataSERP

serp_tool = BrightDataSERP(
    bright_data_api_key=BRIGHT_DATA_API_KEY,
    parse_results=True,
)

In [ ]:
serp_tool.name

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GOOGLE_GEMINI_API_KEY,
)

In [ ]:
from langchain.tools import tool
from langchain.agents import create_agent


@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    return f"Weather in {location}: Sunny, 72°F"


weather_agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant.",
)

results = weather_agent.invoke(
    {"messages": [{"role": "user", "content": "What's the weather in San Francisco?"}]}
)

results["messages"][-1].content

In [ ]:
search_agent = create_agent(
    model=model,
    tools=[serp_tool],
    system_prompt="You are a helpful assistant that can search the internet.",
)

buffer = io.StringIO()

# Prevent output of `result_data` from being displayed because it prints out the Bright Data API key
with contextlib.redirect_stdout(buffer):
    results = search_agent.invoke(
        {"messages": [{"role": "user", "content": "current weather in San Francisco"}]}
    )

results["messages"][-1].content

In [ ]:
for chunk in search_agent.stream(
    {
        "messages": [
            {"role": "user", "content": "search for when Python was first created"}
        ]
    }
):
    agent_result = chunk.get("model")

    if agent_result:
        agent_messages = agent_result.get("messages")

        if not agent_messages:
            continue

        text_response = agent_messages[-1].text

        if text_response:
            print(text_response)